<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_HR_gr1/blob/main/Vyimov%20Zorislav/ZorislavVyymov_Internship_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Блокнот по Стажировке по задаче Оценки кадидатов по резюме.**

## Текущие непонятки по Задаче

### 1. Не ясны критерии оценки "подходит". Подходит для чего. когда. насколько

1. Заказчик представил датасет, размеченный как «подходит»/ «не подходит». Однако, никаких пояснений на тему подходит чему (на какую позицию нужен кандидат, требования к нему/навыкам) не представлено и при этом в запрашиваемых принципах работы в ТЗ Заказчика указано: поддержка в принятии решения "подходит или не подходит человек **на рассматриваемую должность**". Какие должности интересуют Заказчика? Не ясно из чего должна исходить нейронка в этой задаче – она должна «понять что надо Заказчику», основываясь на совокупности всех данных, помеченных как «да»? Но так помечены кандидаты на разную позицию и соответственно обладающие разными и навыками и запросами.  Ожидает ли Заказчик на выходе то же – набора разных кандидатов на разные позиции с разными навыками и запросами? Если да, то как он намерен оценивать результат – ведь предпочтения компании сегодня (когда выдан датасет) отличаются от тех, что будут актуальны через 3 мес (когда будет оценка результатов предсказаний). Если нет – чего ожидает Заказчик? 

### 2. Кто размечал датасет и кто будет это делать в будущем. Используемые принципы

2. Очень интересно кто размечал этот датасет – какими критериями он руководствовался при этом. Какова гарантия того, что эти критерии в тестовой выборке будут такими же, учитывая, что обучаемый датасет будет пополняемым? Если разметчиков несколько – то насколько их взгляды коррелируют - то есть согласен ли каждый со всеми оценками других?

### 3. Как быть со степенью приемлемости кандидата для Заказчика

3. По ТЗ требуется оценить степень приемлемости кандидатов для компании, но разве для этого эта степень должна входить в разметку датасета. Сейчас ее нет - откуда брать? Или эта степень будет выражаться вероятностью результата «подходит»?

### 4. Степень приемлемости кандидата в оценке Заказчика включает отрицательные значения?

4. Если пополняемый датасет будет обогащаться Заказчиком в будущем в том числе и степенью проиемлемости (или предпочтительности(?)) кандидата, то будет ли она включать и отрицательные значения? Если да, то из НС нужно изначально исключать активацию sigmoid? 

Подходящими, возможно, будут linear гипертангенс

### 5. Дообучаемость модели

5. Пока не очень ясно, как обеспечить дообучаемость модели. Видимо, с заказчиком необходимо оговорить формат поставляемых им новых данных, с тем, чтобы автоматически производить их предобработку для подачи в НС с уже используемыми данными. Соответственно, стоит сразу выстраивать автопредобработку данных, применяемой к новым данным. Текущий дататсет можно взять за основу, если только Заказчик не намерен его изменить в будущем. Или получить от заказчика форму датасета в будущем. Если у заказчика нет какого то внятного видения этого будущего представления, возможно лучшим вариантом было бы предложить ему ту форму, которую он обязуется соблюдать при подготовке новых данных и которая максимально соответствовала бы решению задачи. Тогда автопредобработку надо выстраивать именно этого формата.

### 6. Доступность рабочего датасета

Предполагалось, что сотрудник Заказчика, вызодящий на работу с 15.05 предоставит "более расширенный датасет".  Он доступен для работы с ним к н.в.?

## Сборник необходимых для работы библиотек.

In [ ]:
# импортируем keras
from tensorflow import keras
from tensorflow.keras import layers

# Работа с массивами данных
import numpy as np 

# Работа с табличными данными
import pandas as pd

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential, Model

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation, Input, concatenate
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D

# Оптимизаторы
from tensorflow.keras.optimizers import Adam, Adadelta, SGD, Adagrad, RMSprop

# Токенизатор для преобразования текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence

# Масштабирование данных
from sklearn.preprocessing import StandardScaler

# Загрузка датасетов из облака google
import gdown

# Регулярные выражения
import re

# Отрисовка графиков
import matplotlib.pyplot as plt

# Метрики для расчета ошибок
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Для подключения Гугл-диска
from google.colab import drive

# Чтоб рисовать в колабе:
%matplotlib inline


from tensorflow.keras.preprocessing.sequence import pad_sequences #
from tensorflow.keras.callbacks import LambdaCallback # подключаем колбэки

from sklearn.model_selection import train_test_split # Для разбивки на выборки
from sklearn.metrics import mean_squared_error, mean_absolute_error #

#from google.colab import files #  Для загрузки своей картинки

import random #Для генерации случайных чисел 
import math # Для округления

#import os #Для работы с файлами 

import re #

from IPython.display import clear_output

## Исходные данные. Анализ

### Исследование Датасета

Скачаем базу для работы:

In [ ]:
# Чтение файла базы данных из сохраненного как CSV файла:
df = pd.read_csv('DataSet.csv', error_bad_lines=False)

# Вывод количества резюме и числа признаков
print(df.shape)

df.head()

(330, 2)


<ipython-input-3-1b2caa16e32e>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('DataSet.csv', error_bad_lines=False)
Skipping line 4: expected 10 fields, saw 12
Skipping line 5: expected 10 fields, saw 16
Skipping line 6: expected 10 fields, saw 12
Skipping line 8: expected 10 fields, saw 11
Skipping line 10: expected 10 fields, saw 11
Skipping line 14: expected 10 fields, saw 19
Skipping line 17: expected 10 fields, saw 18
Skipping line 19: expected 10 fields, saw 12
Skipping line 20: expected 10 fields, saw 15
Skipping line 26: expected 10 fields, saw 11
Skipping line 30: expected 10 fields, saw 14
Skipping line 33: expected 10 fields, saw 11
Skipping line 34: expected 10 fields, saw 11
Skipping line 37: expected 10 fields, saw 11
Skipping line 39: expected 10 fields, saw 13
Skipping line 40: expected 10 fields, saw 18
Skipping line 42: expected 10 fields, saw 19
Skipping 

,,,,,,,,;Дата отклика;ФИО;Пол,возраст;ЗП;Ищет работу на должность;Город;Занятость;График;Опыт ();Последнее/нынешное место работы;Последняя/нынешняя должность;Образование и ВУЗ;Обновление резюме;Подходит - да/нет
0;23.02.2023;Bessonova Mariya;Женщина;90 000 руб.;Growth Product Manager;Индонезия,Не готова к переезду,не готова к командировкам;проектная работа,частичная занятость,полная занятость;удаленная работа;Опыт работы —5 лет 10 месяцев;PT MYBALI APP GROUP;Менеджер по продукту;Высшее,2023,Нетоло́гия,2014,Университет Российской академии образования,Москва;23 февраля 2023 в 16:30;да
1;11.01.2023;Ефремова Екатерина Вячеславовна;Женщина;не указана;Growth Product Manager;Москва,Готова к переезду,готова к командировкам;полная занятость;удаленная работа,гибкий график,полный день;Опыт работы —11 лет 11 месяцев;Банк Открытие;Product owner трайб сбережения и инвестиции;Высшее,2022,GeekBrains,2010,Московский художественно-промышленный институт,Москва;8 февраля 2023 в 09:15;да
5;25.02.2023;Bessonova Mariya;Женщина,31 год,родилась 26 июня 1991;120 000 руб.;Product manager в Growth Hacking команду;Индонезия,Не готова к переезду,не готова к командировкам;проектная работа,частичная занятость,полная занятость;удаленная работа;Опыт работы —5 лет 11 месяцев;PT MYBALI APP GROUP;Менеджер по продукту;Высшее,2014,Университет Российской академии образования,Москва;23 февраля 2023 в 16:46;да
7;26.01.2023;Маннанова Ксения Дмитриевна;Женщина,24 года,родилась 27 апреля 1998;30 000 руб.;HR-специалист;Уфа,Не готова к переезду,не готова к командировкам;полная занятость;полный день;Опыт работы —5 лет;Tiqum;Контент-менеджер;Среднее специальное,2018,УТЭК;15 февраля 2023 в 10:38;да,NaN,NaN,NaN
9;20.02.2023;Иванова Дарья Дмитриевна;Женщина,28 лет,родилась 5 декабря 1994;не указана;HR-специалист;Уфа,Не готова к переезду,не готова к командировкам;полная занятость;полный день;Опыт работы —4 года 4 месяца;Газпромбанк,ОАО;Специалист;Среднее специальное,2014,Уфимский Политехнический Техникум;1 марта 2023 в 13:26;да,NaN,NaN


Очень много потерянных при загрузке данных, если использовать этот способ загрузки. В данных - каша. Исправлять вручную - не вариант. Использую загрузку оригинального файла в формате .xlsx

---



In [ ]:
# Загрузим как файл .xlsx
data = pd.read_excel('ДатаСет Оценка резюме 581 резюме.xlsx')
data

,Unnamed: 0,Дата отклика,ФИО,"Пол, возраст",ЗП,Ищет работу на должность,Город,Занятость,График,Опыт (),Последнее/нынешное место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Подходит - да/нет
0,0,2023-02-23 00:00:00,Bessonova Mariya,Женщина,90 000 руб.,Growth Product Manager,"Индонезия, Не готова к переезду, не готова к к...","проектная работа, частичная занятость, полная ...",удаленная работа,Опыт работы —5 лет 10 месяцев,PT MYBALI APP GROUP,Менеджер по продукту,"Высшее, 2023, Нетоло́гия, 2014, Университет Ро...",23 февраля 2023 в 16:30,да
1,1,2023-01-11 00:00:00,Ефремова Екатерина Вячеславовна,Женщина,не указана,Growth Product Manager,"Москва, Готова к переезду, готова к командировкам",полная занятость,"удаленная работа, гибкий график, полный день",Опыт работы —11 лет 11 месяцев,Банк Открытие,Product owner трайб сбережения и инвестиции,"Высшее, 2022, GeekBrains, 2010, Московский худ...",8 февраля 2023 в 09:15,да
2,2,2023-02-27 00:00:00,Фролов Артур Михайлович,"Мужчина, 33 года, родился 23 августа 1989",150 000 руб.,Growth Product Manager,"Санкт-Петербург, Готов к переезду: Москва, гот...","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы —7 лет 2 месяца,DIKIY LLC,Директор по маркетингу и бренд-менеджер,"Высшее, 2013, Санкт-Петербургский государствен...",28 февраля 2023 в 09:41,да
3,3,2023-01-11 00:00:00,Хозаев Расул Шагаманович,"Мужчина, 32 года, родился 7 декабря 1990",не указана,Growth Product Manager,"Москва, м. Молодежная, Хочу переехать, готов к...","стажировка, частичная занятость, полная занятость","удаленная работа, гибкий график, полный день",Опыт работы —8 лет 10 месяцев,WeInvest,Product Manager (team leader),"Магистр, 2014, Московский государственный техн...",24 января 2023 в 12:07,да
4,4,2023-01-11 00:00:00,Шадрин Олег Михайлович,"Мужчина, 26 лет, родился 21 февраля 1996",150 000 руб.,Growth Product Manager,"Пермь, Хочу переехать, готов к командировкам","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы —5 лет 9 месяцев,Индивидуальное предпринимательство / частная п...,Product/Project manager,"Высшее, 2021, Российская академия народного хо...",6 февраля 2023 в 11:18,да
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,577,2023-02-28 00:00:00,Логинов Николай Сергеевич,"Мужчина, 24 года, родился 16 мая 1998",70 000 руб.,Project manager,"Москва, м. Китай-город, Уфа, готов к редким ко...",полная занятость,полный день,Опыт работы —3 года 1 месяц,"Алабуга, ОЭЗ ППТ",HR-специалист,"Высшее, 2020, Казанский (Приволжский) федераль...",27 февраля 2023 в 21:27,NaN
575,578,2023-02-17 00:00:00,Шапошников Артём Александрович,"Мужчина, 42 года, родился 27 марта 1980",100 000 руб.,Project manager,"Уфа, Не готов к переезду, готов к командировкам",полная занятость,"удаленная работа, полный день",Опыт работы —12 лет 8 месяцев,Dental Hall,Директор по развитию и связи с общественностью,"Среднее специальное, 2002, УФЭК, 2000, УКСиВТ",15 февраля 2023 в 09:17,NaN
576,579,2023-02-12 00:00:00,Якупов Айрат Юлаевич,"Мужчина, 39 лет, родился 19 октября 1983",80 000 руб.,Project manager,"Уфа, Готов к переезду, готов к командировкам",полная занятость,полный день,Опыт работы —17 лет,"ООО ""Линзы Хойя Рус""","Территориальный менеджер (Уфа, Пермь, Ижевск, ...","Высшее, 2005, Стерлитамакская государственная ...",6 июля 2022 в 15:05,NaN
577,580,2023-01-24 00:00:00,Петрова Лилия Авгатовна,"Женщина, 38 лет, родилась 14 июля 1984",не указана,Product Manager в команду Финансового департам...,"Уфа, Не готова к переезду, готова к редким ком...",полная занятость,"удаленная работа, гибкий график, полный день",Опыт работы —15 лет 9 месяцев,ОАО ФАРМСТАНДАРТ-УФАВИТА,Ведущий экономист,"Высшее, 2006, уфимский технологический институ...",30 января 2023 в 11:56,NaN


Данные загружены и по нумерации строк видно, что фактически данных меньше: в XLS легко определить, что нет строк 541-543. 
Посмотрим на данные, с которыми нам предстоит работать:

In [ ]:
# Вывод количества резюме и числа признаков
print(data.shape)

data.head(3)

(579, 15)


,Unnamed: 0,Дата отклика,ФИО,"Пол, возраст",ЗП,Ищет работу на должность,Город,Занятость,График,Опыт (),Последнее/нынешное место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Подходит - да/нет
0,0,2023-02-23 00:00:00,Bessonova Mariya,Женщина,90 000 руб.,Growth Product Manager,"Индонезия, Не готова к переезду, не готова к к...","проектная работа, частичная занятость, полная ...",удаленная работа,Опыт работы —5 лет 10 месяцев,PT MYBALI APP GROUP,Менеджер по продукту,"Высшее, 2023, Нетоло́гия, 2014, Университет Ро...",23 февраля 2023 в 16:30,да
1,1,2023-01-11 00:00:00,Ефремова Екатерина Вячеславовна,Женщина,не указана,Growth Product Manager,"Москва, Готова к переезду, готова к командировкам",полная занятость,"удаленная работа, гибкий график, полный день",Опыт работы —11 лет 11 месяцев,Банк Открытие,Product owner трайб сбережения и инвестиции,"Высшее, 2022, GeekBrains, 2010, Московский худ...",8 февраля 2023 в 09:15,да
2,2,2023-02-27 00:00:00,Фролов Артур Михайлович,"Мужчина, 33 года, родился 23 августа 1989",150 000 руб.,Growth Product Manager,"Санкт-Петербург, Готов к переезду: Москва, гот...","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы —7 лет 2 месяца,DIKIY LLC,Директор по маркетингу и бренд-менеджер,"Высшее, 2013, Санкт-Петербургский государствен...",28 февраля 2023 в 09:41,да


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579 entries, 0 to 578
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Unnamed: 0                       579 non-null    int64 
 1   Дата отклика                     579 non-null    object
 2   ФИО                              579 non-null    object
 3   Пол, возраст                     579 non-null    object
 4   ЗП                               579 non-null    object
 5   Ищет работу на должность         579 non-null    object
 6   Город                            579 non-null    object
 7   Занятость                        579 non-null    object
 8   График                           579 non-null    object
 9   Опыт ()                          579 non-null    object
 10  Последнее/нынешное место работы  579 non-null    object
 11  Последняя/нынешняя должность     579 non-null    object
 12  Образование и ВУЗ                579

Выводы: 
1. Целевой столбец не содержит данных для 79 записей.  Это вопрос к заказчику (их предоставление)
2. К удалению из ДФ как бесполезные данные для обучения: столбцы 0,1,2,10,13
3. Кажется целесообразным разбить данные S3 на отдельные признаки: "Пол" и "Возраст"
4. Кажется целесообразным разбить данные S12 на отдельные признаки "Образование" и "ВУЗ"

In [ ]:
# посмотрим на единственный цифровой столбец S0:
data.describe()

,Unnamed: 0
count,579.000000
mean,289.196891
std,167.607444
min,0.000000
25%,144.500000
50%,289.000000
75%,433.500000
max,581.000000


Сквозная нумерация строк отличается от нумерации в поле S0. Не хватает 3 строк. Предлагаю удалить этот столбец из DF data

In [ ]:
data.describe(include=['O'])

,Дата отклика,ФИО,"Пол, возраст",ЗП,Ищет работу на должность,Город,Занятость,График,Опыт (),Последнее/нынешное место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Подходит - да/нет
count,579,579,579,579,579,579,579,579,579,579,579,579,579,500
unique,89,507,443,44,8,209,20,30,252,486,358,488,547,2
top,2023-02-22 00:00:00,ФИО скрыто соискателем,Женщина,не указана,Growth Product Manager,"Уфа, Не готова к переезду, не готова к команди...",полная занятость,полный день,Опыт работы —6 лет 9 месяцев,Индивидуальное предпринимательство / частная п...,Product manager,Среднее образование,31 января 2023 в 09:52,нет
freq,48,7,41,288,198,69,342,202,10,12,42,15,3,489


In [ ]:
# посмотрим на столбец "Пол, возраст"
data["Пол, возраст"].value_counts()


Женщина                                       41
Мужчина                                       35
Женщина, 37 лет, родилась 3 января 1986        4
Мужчина, 28 лет, родился 16 августа 1994       4
Мужчина, 21 год, родился 8 марта 2001          3
                                              ..
Женщина, 26 лет, родилась 25 сентября 1996     1
Женщина, 22 года, родилась 17 июля 2000        1
Мужчина, 26 лет, родился 18 апреля 1996        1
Женщина, 27 лет, родилась 11 декабря 1995      1
Женщина, 29 лет, родилась 21 марта 1993        1
Name: Пол, возраст, Length: 443, dtype: int64

выводы: 
1. пол соискателя - обычно значимая информация для работодателя. нужно выводить в самостоятельный атрибут.
2. возраст соискателя - обычно значимая информация для работодателя, но в данном ДФ она неполна для 76 записей. Нужно попытаться выделить ее в самостоятельный атрибут (int) и категоризировать, согласовать с Заказчиком категории. В отношении пустых значений предлагаемого нового атрибута "возраст" предлагаю заместить значением "нет данных"
3. дата рождения соискателя -  обычно не значимая при подборе информация и в данном случае не поможет восполнить недостающие значения атрибута "возраст". По согласованию с Заказчиком предлагаю игнорить эти данные при обучении, т.е. - целиком этот столбец, за исключением извлеченных из него атрибутов п1 и п2

In [ ]:
# Доля пустых значений в полях:
data.isna().sum() / data.shape[0]

Unnamed: 0                         0.000000
Дата отклика                       0.000000
ФИО                                0.000000
Пол, возраст                       0.000000
ЗП                                 0.000000
Ищет работу на должность           0.000000
Город                              0.000000
Занятость                          0.000000
График                             0.000000
Опыт ()                            0.000000
Последнее/нынешное место работы    0.000000
Последняя/нынешняя должность       0.000000
Образование и ВУЗ                  0.000000
Обновление резюме                  0.000000
Подходит - да/нет                  0.136442
dtype: float64

Новой информации это не дало. Разметка Датасета (заполненение подходит/не подходит) является обязательством заказчика - эти данные надо истребовать и перевести в булеву величину.



In [ ]:
# посмотрим на столбец "ЗП"
data["ЗП"].value_counts()

не указана       288
50 000 руб.       27
100 000 руб.      24
70 000 руб.       24
40 000 руб.       23
150 000 руб.      21
60 000 руб.       21
30 000 руб.       20
35 000 руб.       19
80 000 руб.       18
120 000 руб.      15
200 000 руб.       8
45 000 руб.        8
90 000 руб.        7
75 000 руб         5
180 000 руб.       4
110 000 руб.       4
95 000 руб.        3
55 000 руб.        3
85 000 руб.        3
20 000 руб.        3
65 000 руб.        2
1 550 USD          2
130 000 руб.       2
280 000 руб.       2
250 000 руб.       2
1 руб.             2
25 000 руб.        2
2 000 USD          2
100 000 KGS        1
155 000 руб.       1
170 000 руб.       1
3 500 USD          1
600 000 KZT        1
160 000 руб.       1
127 000 руб.       1
119 000 руб.       1
350 000 руб.       1
4 000 USD          1
2 500 USD          1
1 200 EUR          1
1 900 000 KZT      1
140 000 руб.       1
3 000 USD          1
Name: ЗП, dtype: int64

In [ ]:
data["ЗП"].unique()

array(['90 000 руб.', 'не указана', '150 000 руб.', '120 000 руб.',
       '100 000 руб.', '30 000 руб.', '40 000 руб.', '70 000 руб.',
       '50 000 руб.', '140 000 руб.', '130 000 руб.', '80 000 руб.',
       '65 000 руб.', '95 000 руб.', '45 000 руб.', '60 000 руб.',
       '200 000 руб.', '1 900 000 KZT', '1 руб.', '250 000 руб.',
       '280 000 руб.', '110 000 руб.', '600 000 KZT', '1 550 USD',
       '1 200 EUR', '2 500 USD', '85 000 руб.', '4 000 USD',
       '350 000 руб.', '119 000 руб.', '180 000 руб.', '127 000 руб.',
       '160 000 руб.', '2 000 USD', '3 500 USD', '75 000 руб',
       '170 000 руб.', '155 000 руб.', '55 000 руб.', '35 000 руб.',
       '20 000 руб.', '25 000 руб.', '100 000 KGS', '3 000 USD'],
      dtype=object)

Выводы:
1. перевости в единую валюту (руб) и только в числовые значения
2. Значения "1 руб" привести к "не указана" 
3. категорировать значения по согласованию с Заказчиком. 

In [ ]:
data["Город"].value_counts()

Уфа, Не готова к переезду, не готова к командировкам                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              69
Уфа, Не готова к переезду, готова к редким командировкам                                           

Выводы:
1. предлагаю выделить в три отдельных атрибута:
 - текстовый "Город"  и затем категоризировать: (город работодателя (Москва), Миллионники, Зарубеж, остальные)
 - булево "готов к переезду", где 1 принадлежит вариациям "готов к переезду" без конкретики ИЛИ
   "готов к переезду" + город/страна Работодателя;  0 == все остальное
 - 3 категории "готов к командировкам": (нет, редкие, да)

In [ ]:
# рассмотрим содержимое колонки соискателя должности
data["Ищет работу на должность"].unique()

array(['Growth Product Manager',
       'Product manager в Growth Hacking команду',
       'Директор медицинской клиники', 'HR-специалист',
       'Product Manager в команду Финансового департамента',
       'Product Manager в команду HR', 'Project manager',
       'Product Marketing Manager в Growth Hacking команду'], dtype=object)

Вывод: не более 8 вариантов. Категоризировать.

In [ ]:
# рассмотрим содержимое колонки "Занятость"
data["Занятость"].unique()

array(['проектная работа, частичная занятость, полная занятость',
       'полная занятость',
       'стажировка, частичная занятость, полная занятость',
       'полная занятость, частичная занятость, проектная работа',
       'проектная работа, полная занятость',
       'проектная работа, стажировка, волонтерство, частичная занятость, полная занятость',
       'проектная работа, стажировка, частичная занятость, полная занятость',
       'частичная занятость, полная занятость',
       'проектная работа, стажировка, полная занятость',
       'стажировка, полная занятость', 'частичная занятость',
       'проектная работа, неполный рабочий день, полный рабочий день',
       'проектная работа, частичная занятость',
       'полная занятость, частичная занятость, проектная работа, стажировка',
       'проектная работа, волонтерство, частичная занятость, полная занятость',
       'полная занятость, проектная работа',
       'полная занятость, частичная занятость',
       'полная занятость, час

Вывод: 
1. по согласованию с заказчиком исключить вариант "волонтерство", если он ему не интересен/оставить
2. неполный рабочий день == неполная занятость
3. полный рабочий день == полная занятость
4. ввести 4-5 категорий


In [ ]:
# рассмотрим содержимое колонки "График"
data["График"].unique()

array(['удаленная работа', 'удаленная работа, гибкий график, полный день',
       'полный день, гибкий график', 'полный день',
       'удаленная работа, полный день',
       'удаленная работа, гибкий график, полный день, сменный график',
       'вахтовый метод, удаленная работа, гибкий график, полный день, сменный график',
       'удаленная работа, полный день, сменный график',
       'полный день, сменный график', 'удаленная работа, гибкий график',
       'удаленная работа, полный рабочий день',
       'полный день, удаленная работа',
       'полный день, сменный график, гибкий график, удаленная работа',
       'полный день, сменный график, гибкий график, удаленная ',
       'удаленная работа, вахтовый метод',
       'полный день, сменный график, гибкий график',
       'полный день, гибкий график, удаленная работа',
       'полный день, сменный график, гибкий график, вахтовый метод',
       'полный день, гибкий график, удаленная работа, вахтовый метод',
       'полный день, сменный гр

Выводы:
Привести к категориям: вахта, смена, гибкий, полный, удаленка

In [ ]:
# рассмотрим содержимое колонки "Опыт"
data["Опыт ()"].unique()

array(['Опыт работы —5 лет 10 месяцев', 'Опыт работы —11 лет 11 месяцев',
       'Опыт работы —7 лет 2 месяца', 'Опыт работы —8 лет 10 месяцев',
       'Опыт работы —5 лет 9 месяцев', 'Опыт работы —5 лет 11 месяцев',
       'Опыт работы —13 лет 9 месяцев', 'Опыт работы —5 лет',
       'Опыт работы —10 лет 3 месяца', 'Опыт работы —4 года 4 месяца',
       'Опыт работы —13 лет 7 месяцев', 'Опыт работы —5 лет 1 месяц',
       'Опыт работы —16 лет 3 месяца', 'Опыт работы —19 лет 4 месяца',
       'Опыт работы —34 года 6 месяцев', 'Опыт работы —17 лет 6 месяцев',
       'Опыт работы —5 лет 7 месяцев', 'Опыт работы —22 года 3 месяца',
       'Опыт работы —31 год 8 месяцев', 'Опыт работы —23 года 8 месяцев',
       'Опыт работы —12 лет 8 месяцев', 'Опыт работы —23 года 5 месяцев',
       'Опыт работы —24 года 6 месяцев', 'Опыт работы —15 лет 11 месяцев',
       'Опыт работы —13 лет 5 месяцев', 'Опыт работы —19 лет 3 месяца',
       'Опыт работы —20 лет 3 месяца', 'Опыт работы —7 лет 10 месяце

Вывод:
1. Похоже это общий трудовой стаж соискателя
Уточнить: имеет ли отношение к опыту по целевой должности?
2. преобразовать к количеству мес (int)
3. Категоризировать. Категории согласовать с Заказчиком

In [ ]:
# рассмотрим содержимое колонки "Последняя/нынешняя должность"
data["Последняя/нынешняя должность"].unique()

array(['Менеджер по продукту',
       'Product owner трайб сбережения и инвестиции',
       'Директор по маркетингу и бренд-менеджер',
       'Product Manager (team leader)', 'Product/Project manager',
       'Начальник участка инъекционных препаратов', 'Контент-менеджер',
       'Учитель русского языка и литературы', 'Специалист',
       'Ведущий бизнес аналитик', 'Супервайзер отдела продаж',
       'Начальник отдела сопровождения Договоров', 'Бухгалтер',
       'Заместитель начальника управления по налоговому учету и отчетности',
       'Руководитель планово-экономического отдела',
       'Управляющий отделом персонала', 'Финансовый менеджер',
       'Контент-менеджер, продьюсер', 'Начальник отдела казначейства',
       'Главный\xa0 бухгалтер', 'Главный бухгалтер',
       'Финансовый руководитель', 'Главный экономист',
       'Финансовый директор\xa0',
       'Бухгалтер по расчету заработной платы и сдаче отчетности',
       'Агент по туризму', 'Руководитель инвестиционного отдела',


In [ ]:
data["Последняя/нынешняя должность"].nunique()

358

Вывод: почти нет повторений в этом атрибуте. Пока неясно что с этим можно сделать 

In [ ]:
data["Подходит - да/нет"].unique()

array(['да', 'нет', nan], dtype=object)

In [ ]:
data["Подходит - да/нет"].value_counts()

нет    489
да      11
Name: Подходит - да/нет, dtype: int64

Вывод: очень несбалансированный датасет

### Общий вывод и предложения по датасету

Этот дататсет какой то нерабочий. Только 11 подходящих Заказчику резюме из 571 вариантов, 79 из которых вообще не определены ни в какую из категорий. При этом не ясно на какую позицию ищется соискатель (условие в ТЗ "на РАССМАТРИВАЕМУЮ должность". Приобразовывать этот Дататсет и готовить его для обучения кажется нет смысла, тем более что анонсирован другой датасет. Целесообразно дождаться его, возможно он будет иного формата/содержанияи и качества разметки. 

С этим же можно пока потренироваться в обработке, прикинуть что обсудить с Заказчиком.

0. **Индекс**               удалить
1. **Дата отклика**         удалить
2. **ФИО**                  удалить
3. **Пол, возраст**         разделить Пол (булево) и Возраст (кат)
4. **ЗП**                   привести к тыс руб (int), (кат)
5. **Ищет работу на должность** ОНЕ (8)  
6. **Город**                OHE (4) 
7. **Занятость**            кат (4/5)
8. **График**               кат (5) 
9. **Опыт**                 привести к мес (int), (кат)
10. **Последнее/нынешное место работы**   удалить
11. **Последняя/нынешняя должность**      удалить
12. **Образование и ВУЗ**   выделить: Образование (кат) остальное удалить
13. **Обновление резюме**   удалить
14. **Подходит**            кат(2)






## Подготовка исходных данных для обучения.

Присвоение названий столбцам для использования в коде:

In [ ]:
# Настройка названий столбцов. Избавляет от зависимости использования номера по порядку в таблице

COL_SEX_AGE     = data.columns.get_loc('Пол, возраст')
COL_SALARY      = data.columns.get_loc('ЗП')
COL_POS_SEEK    = data.columns.get_loc('Ищет работу на должность')
COL_CITY        = data.columns.get_loc('Город')
COL_EMPL        = data.columns.get_loc('Занятость')
COL_SCHED       = data.columns.get_loc('График')
COL_EXP         = data.columns.get_loc('Опыт ()')
COL_POS_PREV    = data.columns.get_loc('Последняя/нынешняя должность')
COL_EDU         = data.columns.get_loc('Образование и ВУЗ')
COL_UPDATED     = data.columns.get_loc('Обновление резюме')

### Парсинг Датасета

!!! Здесь пока сборник инструментов, их надо адаптировать к Задаче !!!

In [20]:
##############################################
####Убрать кодировку в описаниях \xa0 \n   и лишние столбцы
##############################################
fixed_df = data.drop(data.columns[0],axis = 1)         # Убираем первый столбец с продублированными индексами. Он нам не нужен
fixed_df = fixed_df.drop(fixed_df.columns[0],axis = 1) # Убираем первый столбец с датой отклика. Он нам не нужен
fixed_df = fixed_df.drop(fixed_df.columns[0],axis = 1) # Убираем первый столбец с ФИО. Он нам не нужен
# Вывод количества резюме и числа оставшихся признаков
print(fixed_df.shape)

fixed_df.head(3)


(579, 12)


,"Пол, возраст",ЗП,Ищет работу на должность,Город,Занятость,График,Опыт (),Последнее/нынешное место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Подходит - да/нет
0,Женщина,90 000 руб.,Growth Product Manager,"Индонезия, Не готова к переезду, не готова к к...","проектная работа, частичная занятость, полная ...",удаленная работа,Опыт работы —5 лет 10 месяцев,PT MYBALI APP GROUP,Менеджер по продукту,"Высшее, 2023, Нетоло́гия, 2014, Университет Ро...",23 февраля 2023 в 16:30,да
1,Женщина,не указана,Growth Product Manager,"Москва, Готова к переезду, готова к командировкам",полная занятость,"удаленная работа, гибкий график, полный день",Опыт работы —11 лет 11 месяцев,Банк Открытие,Product owner трайб сбережения и инвестиции,"Высшее, 2022, GeekBrains, 2010, Московский худ...",8 февраля 2023 в 09:15,да
2,"Мужчина, 33 года, родился 23 августа 1989",150 000 руб.,Growth Product Manager,"Санкт-Петербург, Готов к переезду: Москва, гот...","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы —7 лет 2 месяца,DIKIY LLC,Директор по маркетингу и бренд-менеджер,"Высшее, 2013, Санкт-Петербургский государствен...",28 февраля 2023 в 09:41,да


In [15]:
# Убрать кодировку в описаниях \xa0 \n
i=0
for j in range(12):  # Для каждого из столбцов таблицы (их теперь 12)
    for i in range(fixed_df.shape[0]): # По всем резюме
        if type(fixed_df.values[i][j])!=float: # если данные текстовые, то заменяем фрагменты кодировки ASCII на отсутствие символов
            fixed_df.values[i][j] = fixed_df.values[i][j].replace("\xa0","")
            fixed_df.values[i][j] = fixed_df.values[i][j].replace("\n"," ") # здесь замена перевода строки на пробел

In [16]:
# что получилось 

fixed_df.head(3)

,"Пол, возраст",ЗП,Ищет работу на должность,Город,Занятость,График,Опыт (),Последнее/нынешное место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Подходит - да/нет
0,Женщина,90 000 руб.,Growth Product Manager,"Индонезия, Не готова к переезду, не готова к к...","проектная работа, частичная занятость, полная ...",удаленная работа,Опыт работы —5 лет 10 месяцев,PT MYBALI APP GROUP,Менеджер по продукту,"Высшее, 2023, Нетоло́гия, 2014, Университет Ро...",23 февраля 2023 в 16:30,да
1,Женщина,не указана,Growth Product Manager,"Москва, Готова к переезду, готова к командировкам",полная занятость,"удаленная работа, гибкий график, полный день",Опыт работы —11 лет 11 месяцев,Банк Открытие,Product owner трайб сбережения и инвестиции,"Высшее, 2022, GeekBrains, 2010, Московский худ...",8 февраля 2023 в 09:15,да
2,"Мужчина, 33 года, родился 23 августа 1989",150 000 руб.,Growth Product Manager,"Санкт-Петербург, Готов к переезду: Москва, гот...","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы —7 лет 2 месяца,DIKIY LLC,Директор по маркетингу и бренд-менеджер,"Высшее, 2013, Санкт-Петербургский государствен...",28 февраля 2023 в 09:41,да


#### Сборник готовых функций

In [17]:
# Данные о поле и возрасте
def getParameterSexAge(arg):
  out = [0,0]
  #Если М, то 1. По умолчанию 0 - Ж
  if "М" in arg:
    out[0] = 1
  # текущий год - год рождения
  year_tec = 2020
  if (len(arg) > 7):
    out[1] = year_tec - int(re.findall(r'\d{4}', arg)[0])
  return out

# Полученный возраст превращаем в класс возрастной категории
def getParameterAgeVect(arg):
  outClass =int((arg-13)/5)
  outClass = max(0, min(10,outClass))
  # На выходе получаем вектор с нужной категорией возраста
  return list(utils.to_categorical(outClass, 11).astype('int'))


# Зарплата
def getParameterSalary(arg):
    num = arg
    # Сначала получаем чистое число, убираем лишние знаки
    if (type(num) == str):
        num = re.sub(' ','',num)
        num = re.sub('[а-яА-ЯёЁ]','',num)
        num = re.sub('[a-zA-Z]','',num)
        num = num.replace('.','')
        
        # Получаем чисто валюту, убираем цифры
        curr = re.sub('[0-9]','',arg)
        curr = curr.replace('.','').strip()
        
        # Конвертируем в рубли, если валюта
        if curr == 'USD':
           num = float(num)*65
        elif curr == 'KZT':
           num = float(num)*0.17
        elif curr=='грн':
           num = float(num)*2.6
        elif curr=='белруб':
           num = float(num)*30.5
        elif curr=='EUR':
           num = float(num)*70
        elif curr=='KGS':
           num = float(num)*0.9
        elif curr=='сум':
           num = float(num)*0.007
        elif curr=='AZN':
           num = float(num)*37.5
             
    salaryStr = int(num) 

    
    return salaryStr


# Данные о городе
def getParameterCity(arg):
  millionCities = "Новосибирск Екатеринбург Нижний Новгород Казань Челябинск Омск Самара Ростов-на-Дону Уфа Красноярск Пермь Воронеж Волгоград"
  sarg=arg.split(',')
  for item in sarg:
    item= item.strip()
    if item == "Москва":          return [1, 0, 0, 0]
    if item == "Санкт-Петербург": return [0, 1, 0, 0]
    if item in millionCities:     return [0, 0, 1, 0]
  return [0, 0, 0, 1]

# Данные о желаемой занятости
def getParameterEmployment(arg):
  out= [0, 0, 0, 0]
  if ("стажировка" in arg):           out[0]=1
  if ("частичная занятость" in arg):  out[1]=1
  if ("проектная работа" in arg):     out[2]=1
  if ("полная занятость" in arg):     out[3]=1
  return out

# Данные о желаемом графике работы
def getParameterSchedule(arg):
  out = [0, 0, 0, 0]
  if ("гибкий график" in arg):    out[0]=1
  if ("полный день" in arg):      out[1]=1
  if ("сменный график" in arg):   out[2]=1
  if ("удаленная работа" in arg): out[3]=1
  return out

# Данные об образовании
def getParameterEducation(arg):
  out = [0, 0, 0, 0] #По умолчанию не указано
  if "Высшее" in arg or "Higher education" in arg:  out[0] = 1
  if "Среднее специальное" in arg:      out[1] = 1
  if "Неоконченное высшее" in arg:      out[2] = 1
  if "Среднее образование" in arg:      out[3] = 1
  return out

# Данные об опыте работы
def getParameterExperience(arg):
  arg = str(arg)
  #Проверяем, если не пустая строка
  symbols = 0
  years = 0
  months = 0
  for s in arg:
    if (s != " "):
      symbols += 1
  
  #Находим индексы пробелов около фразы "опыт работы"
  if (symbols > 10):
      spacesIndexes = []
      index = 0
      while ((len(spacesIndexes) < 5) & (index < len(arg))):
          if (arg[index] == " "):
              spacesIndexes.append(index)
          index += 1

      years = 0
      months = 0
      if (arg[spacesIndexes[2]+1] != "м"):
         if (len(spacesIndexes) >= 3):
            yearsStr = arg[spacesIndexes[1]:spacesIndexes[2]] # Записываем в строку значение лет
            years = int(yearsStr)
      
         if (len(spacesIndexes) >= 5):
            monthsStr = arg[spacesIndexes[3]:spacesIndexes[4]] # Записываем в строку значение месяцев
            if(arg[spacesIndexes[2]+1] == "м"):
                months = int(monthsStr)
      else:
        if (len(spacesIndexes) >= 3):
          monthsStr = arg[spacesIndexes[1]:spacesIndexes[2]]
          months = int(monthsStr)
      
  return 12*years+months

# Категориальное представление опыта работы
def getParameterExperienceVector(arg):
  out = getParameterExperience(arg)
  outClass = 0
  if (out > 6): # если больше 6 месяцев
    outClass = 1
  if (out > 12): # если больше 12 месяцев
    outClass = 2
  if (out > 24): # если больше 24 месяцев
    outClass = 3
  if (out > 36): # если больше 36 месяцев
    outClass = 4
  if (out > 60): # если больше 60 месяцев
    outClass = 5
  if (out > 96): # если больше 96 месяцев
    outClass = 6
  if (out > 120): # если больше 120 месяцев
    outClass = 7
  if (out > 156): # если больше 156 месяцев
    outClass = 8
  if (out > 192): # если больше 192 месяцев
    outClass = 9 
  if (out > 240): # если больше 240 месяцев
    outClass = 10
  
  return list(utils.to_categorical(outClass, 11).astype('int'))



#### Применение функций для извлечения данных 

In [18]:
# Извлекаем все параметры
def getAllParameters(val):
  result = getParameterSexAge(val[0])
  sex = result[0]  #getParameterSex() #параметры о поле
  age =getParameterAgeVect(result[1]) #параметры о возрасте
  city = getParameterCity(val[3]) #параметры о городе
  employment = getParameterEmployment(val[4]) #параметры о желаемой занятости
  shedule = getParameterSchedule(val[5]) #параметры о желаемом графике
  education = getParameterEducation(val[9]) #параметры об образовании
  experience = getParameterExperienceVector(val[6]) #параметры об опыте
  out = [] 
  # Склеиваем все параметры в вектор  
  out.append(sex)
  out += age
  out += city
  out += employment
  out += shedule
  out += education
  out += experience
  
  return out

In [19]:
# Создаем тренировочную выборку
def get01Data(values):
  xTrain = []
  yTrain = []
  
  # Если предсказывать будем зарплату:
  for val in values:
    y = getParameterSalary(val[1])
    x = getAllParameters(val)
    xTrain.append(x)
    yTrain.append(y/1000)
  
  xTrain = np.array(xTrain)
  yTrain = np.array(yTrain)

  
  return (xTrain, yTrain)

## Построение модели первой точности

Это свалка, надо разобрать.

###  Вариант 1

Для задачи регрессии, где требуется оценивать пригодность кандидата на основе табличных данных из резюме, можно использовать нейронную сеть с выходным слоем, который будет предсказывать числовое значение. Однако, если на выходе должно быть только оценка в форме "подходит" или "не подходит", можно преобразовать в задачу бинарной классификации, где выходной слой будет иметь один нейрон с сигмоидальной активацией.

Вариант кода, для создания простуй нейронной сети для решения данной задачи:

In [ ]:
# Предварительно: Загрузка и предобработка данных
# X_train - входные данные (табличные параметры из резюме)
# y_train - выходные данные (оценка пригодности)

# Создание модели нейронной сети из предположения, что на входе 10 значимых параметров резюме
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(10,)) 
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Компиляция модели
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Оценка модели на тестовых данных
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

# Прогнозирование пригодности новых данных
new_data = np.array([[...]])  # новые данные (табличные параметры резюме)
prediction = model.predict(new_data)
if prediction >= 0.5:
    print("Подходит")
else:
    print("Не подходит")


### Вариант 2 (предложен ChatGPT)

Для задачи регрессии по описанному датасету с табличными данными из резюме кандидатов, можно рассмотреть использование следующей модели нейронной сети:
Модель: Многослойный перцептрон (Multilayer Perceptron, MLP)
Многослойный перцептрон (MLP) является одним из базовых типов нейронных сетей. Он состоит из нескольких слоев нейронов, включая входной слой, скрытые слои и выходной слой. Каждый нейрон в слоях связан с нейронами в следующем слое и обрабатывает входные данные с помощью активационной функции.
Пример кода для создания модели MLP с использованием библиотеки Keras:


In [ ]:
#  Вариант предложенный Chat GPT:


# Определение модели MLP
model = Sequential()
model.add(Dense(64, input_dim=10, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))

# Компиляция модели
model.compile(loss='mean_squared_error', optimizer='adam')

# Обучение модели
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Оценка модели
mse = model.evaluate(X_test, y_test)


В этом примере модель состоит из трех слоев: входного слоя с 10 нейронами, двух скрытых слоев с 64 нейронами каждый и выходного слоя с одним нейроном. В качестве функции активации используется ReLU для скрытых слоев и линейная функция активации для выходного слоя.
Модель компилируется с использованием функции потерь "mean_squared_error" (средняя квадратичная ошибка) и оптимизатора "adam". Затем модель обучается на тренировочных данных с заданным количеством эпох и размером пакета.
В конце модель оценивается на тестовых данных, и среднеквадратичная ошибка (MSE) используется для оценки производительности модели.
Вы можете настроить параметры модели, такие как количество скрытых слоев, количество нейронов в каждом слое, функции активации и оптимизатор, чтобы достичь оптимальных результатов в вашей конкретной задаче.


### Вариант 3 с обчающего ноутбука от Кузина С.

#### Нейронка по простым данным

*Разбор данного раздела:* https://youtu.be/h8cOCvlYmN8?t=964

In [ ]:
# Обучаем модель полученными данными
model = Sequential()
model.add(BatchNormalization(input_shape=(xTrain01.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(1000, activation='tanh'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer=Adam(lr=1e-5), loss='mse', metrics=['mae'])

history = model.fit(xTrain01, 
                    yTrain, 
                    epochs=50, 
                    batch_size=32,
                    validation_split=0.15, 
                    verbose=2)

plt.plot(history.history['mae'], 
         label='Средняя абсолютная ошибка на обучающем наборе')
plt.plot(history.history['val_mae'], 
         label='Средняя абсолютная ошибка на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Средняя абсолютная ошибка')
plt.legend()
plt.show()

In [ ]:
# Делаем предсказание и приводим его к начальному виду
pred = model.predict(xTrain01)
pred = pred.flatten()
# Среднее значение ошибки 
delta = pred - yTrain
absDelta = abs(delta)
print(sum(absDelta) / len(absDelta))

In [ ]:
# В качестве нормализации данных используем готовую библиотеку 
yScaler = StandardScaler()

# Скармливаем объекту двумерный вектор
yScaler.fit(yTrain.reshape(-1, 1))

# Нормализуем по нормальному распределению
yTrainScaled = yScaler.transform(yTrain.reshape(-1, 1))

print(yTrainScaled.shape)
print(yTrain[1])
print(yTrainScaled[1])

In [ ]:
print(min(yTrain), max(yTrain))
print(min(yTrainScaled), max(yTrainScaled))

In [ ]:
# Обучаем модель стандартизированными данными
modelS = Sequential()
modelS.add(BatchNormalization(input_shape=(xTrain01.shape[1],)))
modelS.add(Dense(128, activation='relu'))
modelS.add(Dense(1000, activation='tanh'))
modelS.add(Dense(100, activation='relu'))
modelS.add(Dense(1, activation='linear'))

modelS.compile(optimizer=Adam(lr=1e-5), loss='mse', metrics=['mae'])

history = modelS.fit(xTrain01, 
                    yTrainScaled, 
                    epochs=50, 
                    batch_size=64,
                    validation_split=0.15, 
                    verbose=2)

plt.plot(history.history['mae'], 
         label='Средняя абсолютная ошибка на обучающем наборе')
plt.plot(history.history['val_mae'], 
         label='Средняя абсолютная ошибка на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Средняя абсолютная ошибка')
plt.legend()
plt.show()

In [ ]:
# Делаем предсказание и приводим его к начальному виду
pred = modelS.predict(xTrain01)
predUnscaled = yScaler.inverse_transform(pred).flatten()

# Среднее значение ошибки 
delta = predUnscaled - yTrain
absDelta = abs(delta)
print(sum(absDelta) / len(absDelta))

#### Оценка результатов

*Разбор данного раздела:* https://youtu.be/h8cOCvlYmN8?t=1314



In [ ]:
yy = model.predict(xTrain01) # Результат для всего тренировочного набора

In [ ]:
n = 10
for i in range(n):
  print('Реальное значение - ',yTrain[i], " Предсказанное значение - ", yy[i][0], " Разница - ", abs(yTrain[i]-yy[i][0]))

### План подготовки данных

 Для полноценного обучения модели требуется предварительная обработка данных, разделение на тренировочный и тестовый наборы, а также возможно масштабирование и др. 

Примерный план подготовительных шагов в подготовке данных для обучения модели:

1. Задать понятные названия столбцам для использования в коде без индексов.

2. Предобработка данных: Проверить данные на наличие пропущенных значений и выбросов. Может потребоваться заполнить пропущенные значения или удалить аномальные данные. Также может потребоваться выполнить масштабирование или нормализацию признаков, чтобы обеспечить стабильность обучения модели.

3. Разделение на тренировочный и тестовый наборы. Обычно используется отношение 80/20 или 70/30. Но у нас очень незначительное количество данных - всего 800 строк. Похоже придется 85/15 делать.

4. Кодирование целевой переменной: Поскольку целевая переменная ("подходит" или "не подходит") представлена в виде текстовых меток, потребуется кодирование для преобразования ее в числовой формат. Например, можно присвоить числу 1 для "подходит" и числу 0 для "не подходит".

5. Обработка дисбаланса классов: Навскидку, исходная БД очень несбалансирована.Потребуется изучить и применить методы балансировки классов, такие как взвешивание классов или аугментация текстовых/табличных данных, чтобы предотвратить смещение модели в сторону преобладающего класса. 

6. Гиперпараметры и кросс-валидация: Нужно определить оптимальные значения гиперпараметров модели, такие как количество нейронов, количество слоев, скорость обучения и другие. Нужно изучить и попробовать использовать методы кросс-валидации для оценки производительности модели на различных подмножествах данных и выбора лучших параметров.

7. Проверка настройки модели в соответствии с требованиями к точности предсказаний и обработанными данными перед началом обучения на обещанных боевых данных от Заказчика.

8. Сборка фрейма предобработки данных от Заказчика для подачи на обучение модели.